#### 1. Setup

In [ ]:
%pip install pandas numpy seaborn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk

In [4]:
reviews_amazon = pd.read_csv(r'Reviews_Amazon.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Reviews_Amazon.csv'

In [ ]:
reviews_amazon

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [ ]:
reviews_amazon.drop(axis=1,
                    columns=['ProductId', 'UserId', 'Time', 'Summary',
                             'ProfileName',	'HelpfulnessNumerator',
                             'HelpfulnessDenominator'],
                    inplace=True)
reviews_amazon

,Id,Score,Text
0,1,5,I have bought several of the Vitality canned d...
1,2,1,Product arrived labeled as Jumbo Salted Peanut...
2,3,4,This is a confection that has been around a fe...
3,4,2,If you are looking for the secret ingredient i...
4,5,5,Great taffy at a great price. There was a wid...
...,...,...,...
568449,568450,5,Great for sesame chicken..this is a good if no...
568450,568451,2,I'm disappointed with the flavor. The chocolat...
568451,568452,5,"These stars are small, so you can give 10-15 o..."
568452,568453,5,These are the BEST treats for training and rew...


In [ ]:
# A tabela é muito grande para rodar em uma CPU, por isso usaremos
# parte somente parte dela como exemplo
reviews_amazon = reviews_amazon[:500]

#### 2. Classificando as reviews com o RoBERTa

<br>

#### RoBERTa - Facebook (default)

##### 1. Na tokenização, as subwords tem como base os bytes ao invés dos caracteres unicode, característica também do GPT-2, isso aumenta o vocabulário disponível do BERT padrão em aproximadamente 20 mil palavras, ajudando o modelo a compreender palavras raras.
#### 2. O modelo RoBERTa é treinado em uma combinação de conjuntos de dados que totalizam 160 GB, enquanto o BERT é treinado inicialmente em 13 GB.

<br>

##### RoBERTa - Twitter (pretrained for Sentiment Analysis)

##### 1. Pré-treinado em cerca de 120 milhões de posts do twitter em inglês
##### 2. Tweets foram classificados e separados em 3 grupos: negativo, neutro, positivo

In [ ]:
%pip install transformers scipy

In [ ]:
import torch

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from tqdm import tqdm

c:\Users\selma\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Definindo o modelo RoBERTa
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"

# Obtendo o tokenizador e os pesos do modelo escolhido
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
# Função que irá tokenizar o texto da review e analisá-la sentimentalmente
def polarity_scores_RoBERTa(review_text, model_op) -> dict:

    def apply_label(neg, neu, pos):
        if max(neg, neu, pos) == neg:
            return '-1'
        elif max(neg, neu, pos) == neu:
            return '0'
        else:
            return '1'

    def defineModel(model_op):

        if model_op:
            model_name = f"cardiffnlp/twitter-roberta-base-sentiment"
        else:
            model_name = f'FacebookAI/roberta-base'

        return model_name

    # Definindo o modelo
    model_name = defineModel(model_op)

    # Obtendo o tokenizador e os pesos do modelo escolhido
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)

    # is in charge of preparing the inputs in the appropriate format
    # for the model — tensors
    tokenized_text = tokenizer(review_text, return_tensors='pt')

    output = model(**tokenized_text)

    scores = softmax(output[0][0].detach().numpy())

    # One model has neg, neu and pos, but the other has
    # only to values: neg e pos
    if model_op:
        neg, neu, pos = scores
    else:
        neu = 0
        neg, pos = scores

    return {'RoBERTa_neg' : neg,
            'RoBERTa_neu' : neu,
        'RoBERTa_pos' : pos,
            'Label' : apply_label(neg, neu, pos)}

In [ ]:
def classifyReviews(model_op: bool, reviews : pd.DataFrame):

    def createDataframe(results, reviews):

        df = pd.DataFrame(results).T
        df = df.reset_index().rename(
            columns={'index': 'Id'})

        return df.merge(reviews, how='left')

    # Archiving the results of the model picked
    model_results = {}

    # O tqdm está sendo usado para mostrar uma barra de progresso
    # enquanto o loop itera sobre as linhas do DataFrame
    for i, row in tqdm(reviews.iterrows(), total=len(reviews)):
        try:
            text = row['Text']
            myid = row['Id']
            model_result = polarity_scores_RoBERTa(text, model_op)
            model_results[myid] = model_result
        except RuntimeError:
            print(f'Broke for id {myid}')

    # Formatando o dataframe
    return createDataframe(model_results, reviews)

In [ ]:
#roberta_results_df = classifyReviews(True, reviews_amazon)
roberta_default_results_df = classifyReviews(False, reviews_amazon)
roberta_results_df = classifyReviews(True, reviews_amazon)

##### 2.1 Comparando os modelos

In [ ]:
# Analisando linha por linha, criaremos um novo dataframe que armazenará
# a diferença entre os dois modelos testados
def compareModels(df1, model1 : str, df2, model2 : str):

    # Os dataframes devem ser do mesmo tamanho obrigatoriamente
    assert len(df1) == len(df2)

    comparisons = dict()

    for id in range(len(df1)):

        row1 = df1.iloc[id]
        row2 = df2.iloc[id]

        label1 = row1['Label']
        label2 = row2['Label']

        if label1 != label2:
            myid, text = row1['Id'], row1['Text']

            diff = {'Label' + model1: label1,
            'Label' + model2: label2,
            'Text': text}

            comparisons[myid] = diff

    return pd.DataFrame(comparisons).T.reset_index().rename(
            columns={'index': 'Id'})

In [ ]:
diff_models_df = compareModels(roberta_results_df, 'RoBERTa-Twitter',
             roberta_default_results_df, 'RoBERTa-Default')
diff_models_df.head()

In [ ]:
roberta_results_df.head()

In [ ]:
roberta_default_results_df.head()

In [ ]:
# Abstração de saída de dados
roberta_results_df.to_csv(r'roberta_results.csv',
                          sep=';',
                          index=False)
roberta_default_results_df.to_csv(r'roberta_default_results.csv',
                          sep=';',
                          index=False)

#### 3. Visualização dos Dados

##### 3.1 Distribuição das reviews

In [ ]:
def showPercents(df : pd.DataFrame):
    # Conta as ocorrências das labels e normaliza para que
    # representem porcentagens
    label_counts = df['Label'].value_counts(normalize=True) * 100

    # Criando a figura do df para posteriormente passá-la pra png
    fig, ax = plt.subplots()
    label_counts.plot(kind='bar', color='lightgreen', ax=ax)

    # Adicionando as porcentagens nas barras
    for i in ax.patches:
        ax.annotate(f'{i.get_height():.2f}%', 
                    (i.get_x() + i.get_width() / 2, i.get_height()), 
                    ha='center', va='baseline', fontsize=10,
                    color='black', xytext=(0, 5),
                    textcoords='offset points')

    # Detalhes da imagem
    plt.title('Distribuição das Labels')
    plt.xlabel('Label')
    plt.ylabel('Porcentagem (%)')

    # Salvando a figura
    plt.savefig('label_distribution.png', bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
showPercents(roberta_results_df)

##### 3.2 Nuvem de Palavras

In [ ]:
from wordcloud import WordCloud

In [ ]:
def showWordCloud(df : pd.DataFrame):
    # Junta todos os reviews em um único texto
    text = " ".join(review for review in df['Text'])

    # Gere a word cloud
    wordcloud = WordCloud(max_font_size=50, max_words=100,
                          background_color="white").generate(text)

    plt.figure()
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()

In [ ]:
def splitDataframe(df : pd.DataFrame):
    pos = df[df['Label'] == '1']
    neg = df[df['Label'] == '-1']
    neu = df[df['Label'] == '0']

    return pos, neg, neu

In [ ]:
# Divindo o dataframe com base nas labels
df_pos, df_neg, df_neu = splitDataframe(roberta_results_df)

In [ ]:
# Mostrando uma nuvem de palavras com as reviews
showWordCloud(df_neg)

In [ ]:
showWordCloud(df_neu)

In [ ]:
showWordCloud(df_pos)

##### 3.3 Scores x Pontuações

In [ ]:
def showScoresDistribution(df : pd.DataFrame):

    scores = df['Score'].value_counts().sort_index()

    fig, ax = plt.subplots(figsize=(7, 4))
    scores.plot(kind='bar',
            title='Quantity of reviews by Stars', color='yellow', ax=ax)

    ax.set_xlabel('Review Stars')

    plt.show()

In [ ]:
def showScoresComparison(df : pd.DataFrame):

    fig, axs = plt.subplots(1, 3, figsize=(11, 4))

    sns.barplot(data=df, x='Score', y='RoBERTa_pos',
                color='green', ax=axs[0])
    sns.barplot(data=df, x='Score', y='RoBERTa_neg',
                color='red', ax=axs[1])
    sns.barplot(data=df, x='Score', y='RoBERTa_neg',
                color='orange', ax=axs[2])

    axs[0].set_title('Positive')
    axs[1].set_title('Neutral')
    axs[2].set_title('Negative')
    plt.tight_layout()

plt.show()

In [ ]:
showScoresDistribution(roberta_results_df)

In [ ]:
showScoresComparison(roberta_results_df)

#### 4. Usando o GPT-3 para classificar as reviews

In [ ]:
%pip install openai==0.28

In [ ]:
import openai

In [1]:
# openai.api_key = ?

In [ ]:
def classifyReviewsGPT(reviews : pd.DataFrame):

    gpt_results = dict()

    for i, row in tqdm(reviews.iterrows(), total=len(reviews)):
        try:
            text = row['Text']
            myid = row['Id']

            prompt = f"Classifique a seguinte review como positivo (1),\
            neutro (0) ou negativo (-1): \n\n Review: {text}\
             \n Classificação:"

            response = openai.Completion.create(
                engine="gpt-3.5-turbo-instruct",
                prompt=prompt,
                max_tokens=20,
                n=1,
                stop=None,
                temperature=0.5)

            gpt_result = response.choices[0].text
            gpt_results[myid] = gpt_result

        except RuntimeError:
            print(f'Broke for id {myid}')

    return createDataframe(gpt_results, reviews)

In [ ]:
gpt_results_df = classifyReviewsGPT(reviews_amazon)

##### 4.1 Comparando o gpt com outros modelos

In [ ]:
diff_models_df = compareModels(roberta_results_df, 'RoBERTa-Twitter',
                               gpt_results_df, 'GPT-3')
diff_models_df

In [ ]:
diff_models_df = compareModels(roberta_default_results_df, 'RoBERTa-Default',
                               gpt_results_df, 'GPT-3')
diff_models_df

#### 5. Integração Web Scraping e Modelos

In [ ]:
reviews_produto = pd.read_csv(r'/content/drive/MyDrive/Projeto EDL/Reviews Sentiment Analysis/the_ship_reviews.csv')

In [ ]:
reviews_produto = reviews_produto[:100]
reviews_produto.head()

In [ ]:
produto_gpt = classifyReviewsGPT(reviews_produto)

In [ ]:
produto_gpt.head()

In [ ]:
produto_roberta_df = classifyReviews(False, reviews_produto)
produto_roberta_twitter = classifyReviews(True, reviews_produto)

In [ ]:
produto_gpt.to_csv(r'/content/drive/MyDrive/Projeto EDL/Reviews Sentiment Analysis/gpt_results.csv',
                          sep=';',
                          index=False)

In [ ]:
diff = compareModels(produto_gpt, 'GPT-3',
                     produto_roberta_twitter, 'RoBERTa-Twitter')
diff